<a href="https://www.bigdatauniversity.com"><img src = "https://is4-ssl.mzstatic.com/image/thumb/Purple113/v4/91/42/8b/91428ba5-e783-cce6-767b-e023b2db39dc/AppIcon-0-1x_U007emarketing-0-7-0-85-220.png/246x0w.png" width = 200, align = "center"></a>
<h1 align=center><font size=5>Yahoo Finance Web scraping</font></h1>

<h2>Table of content</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a>Web scrapping : one ticker</a>
        <ul>
            <li><a>Identify_classes and attributs</a></li>
            <li><a>Convertion into dataframe</a></li>
    </ul>
    </li>
    <li><a>Web scrapping : Multi tickers</a></li>
    <li><a>Web scrapping : Potential Improvements</a></li>

Estimated Time Needed: <strong>5 min</strong>
</div>
 
<hr>

<h1>What is the purpose of this tool?</h1>

This tool is a way to experiment web scrapping on yahoo finance website.The aim is to be able to scrap the approriate attributes upon which an analysis could be made.

Before hand let's import all the packages needed

In [188]:
import tkinter as tk
from tkinter import simpledialog
from bs4 import BeautifulSoup
import json
import os
from urllib.request import Request, urlopen

<h2>Web scrapping : one ticker</h2>

First the user will have to input the ticker of the yahoo web page (Tkinter)<p>
Knowing that the url is the same for equity instruments , the ticker is the only variable changing.

In [189]:
ROOT = tk.Tk()
ROOT.withdraw()
# the input dialog
ticker = simpledialog.askstring(title="Ticker",
                                  prompt="Please enter ur ticker")
url = f'https://fr.finance.yahoo.com/quote/{ticker}?p={ticker}'

Second we’ll use a Python module called BeautifulSoup, the most common web scraping module for Python.

In [190]:
req = Request(url)
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
html

b'<!DOCTYPE html>\n<html class="NoJs featurephone" id="atomic" lang="fr-FR">\n <head prefix="og: http://ogp.me/ns#">\n  <script>\n   window.performance && window.performance.mark && window.performance.mark(\'PageStart\');\n  </script>\n  <meta charset="utf-8"/>\n  <title>\n   Amazon.com, Inc. (AMZN) - Cours des actions, cotation, historique et actualit\xc3\xa9s - Yahoo\xc2\xa0Finance\n  </title>\n  <meta content="AMZN, Amazon.com, Inc., AMZN graphique du cours des actions, Amazon.com, Inc. graphique du cours des actions, graphique du cours des actions, actions, cotations, finance" name="keywords"/>\n  <meta content="on" http-equiv="x-dns-prefetch-control"/>\n  <meta content="on" property="twitter:dnt"/>\n  <meta content="141301389258994" property="fb:app_id"/>\n  <meta content="#400090" name="theme-color"/>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <meta content="Recherchez les derni\xc3\xa8res actualit\xc3\xa9s, les cotations boursi\xc3\xa8res et l\xe

The result is messy , we need to figure out how to find the attributes we need in this bulk of non-parsed information. 

<h3>Identify classes and attributes </h3>

Often, the distinctive mark resides in the class attribute (in our case 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)).We inspect the HTML lines from the source code of the website , we’ll notice a pattern. 

Now we’ll extract, by turn, each item of interest and store it in a json file:

- PRESENT_VALUE.
- PRESENT_GROWTH.
- PREV_CLOSE.
- OPEN.
- ASK.
- BID.
- DAYS_RANGE.
- FIFTY_TWO_WK_RANGE.
- TD_VOLUME.
- AVERAGE_VOLUME_3MONTH
- MARKET_CAP
- BETA_3Y
- PE_RATIO
- EPS_RATIO
- EARNINGS_DATE
- EARNINGS_DATE
- DIVIDEND_AND_YIELD
- EX_DIVIDEND_DATE
- ONE_YEAR_TARGET_PRICE

In [191]:
company_json = {}
other_details = {}
for span in soup.findAll('span',
                         attrs={'class': 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'
                         }):
    other_details['PRESENT_VALUE'] = span.text.strip()
for div in soup.findAll('div', attrs={'class': 'D(ib) Va(t)'}):
    for span in div.findAll('span', recursive=False):
        other_details['PRESENT_GROWTH'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['PREV_CLOSE'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['OPEN'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['BID'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['ASK'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['DAYS_RANGE'] = span.text.strip()
for td in soup.findAll('td',
                       attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['FIFTY_TWO_WK_RANGE'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['TD_VOLUME'] = span.text.strip()
for td in soup.findAll('td',
                       attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'
                       }):
    for span in td.findAll('span', recursive=False):
        other_details['AVERAGE_VOLUME_3MONTH'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['MARKET_CAP'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['BETA_3Y'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['PE_RATIO'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['EPS_RATIO'] = span.text.strip()
for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'
                       }):
    other_details['EARNINGS_DATE'] = []
    for span in td.findAll('span', recursive=False):
        other_details['EARNINGS_DATE'].append(span.text.strip())
for td in soup.findAll('td',
                       attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
    other_details['DIVIDEND_AND_YIELD'] = td.text.strip()
for td in soup.findAll('td',
                       attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
    for span in td.findAll('span', recursive=False):
        other_details['EX_DIVIDEND_DATE'] = span.text.strip()
for td in soup.findAll('td',
                       attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'
                       }):
    for span in td.findAll('span', recursive=False):
        other_details['ONE_YEAR_TARGET_PRICE'] = span.text.strip()
company_json['OTHER_DETAILS'] = other_details
with open('data.json', 'w') as outfile:
    json.dump(company_json, outfile, indent=4)
print (company_json)
with open('output_file.html', 'wb') as file:
    file.write(html)
print ('----------Extraction of data is complete. Check json file.----------')


{'OTHER_DETAILS': {'PRESENT_VALUE': '2\xa0401,10', 'PREV_CLOSE': '2\xa0410,39', 'OPEN': '2\xa0384,33', 'BID': '2\xa0402,71 x 1000', 'ASK': '2\xa0404,90 x 1200', 'TD_VOLUME': '3\xa0092\xa0643', 'AVERAGE_VOLUME_3MONTH': '6\xa0027\xa0435', 'MARKET_CAP': '1,198T', 'PE_RATIO': '114,69', 'EPS_RATIO': '20,94', 'EARNINGS_DATE': ['23 juil. 2020', '27 juil. 2020'], 'DIVIDEND_AND_YIELD': 'S.O. (S.O.)', 'ONE_YEAR_TARGET_PRICE': '2\xa0645,67'}}
----------Extraction of data is complete. Check json file.----------


<h3>Convertion into dataframe </h3>

In [192]:
import pandas as pd  
data = pd.read_json("data.json") 
data.rename(columns={"OTHER_DETAILS": "Value"},inplace=True)
data

,Value
ASK,"2 404,90 x 1200"
AVERAGE_VOLUME_3MONTH,6 027 435
BID,"2 402,71 x 1000"
DIVIDEND_AND_YIELD,S.O. (S.O.)
EARNINGS_DATE,"[23 juil. 2020, 27 juil. 2020]"
EPS_RATIO,"20,94"
MARKET_CAP,"1,198T"
ONE_YEAR_TARGET_PRICE,"2 645,67"
OPEN,"2 384,33"
PE_RATIO,"114,69"


<h2>Web scrapping : Multi tickers</h2>

Using the same reasoning , I've given the chance to the user to fetch multitude of tickers. The user has to input tickers separated by comma.

In [185]:
ROOT = tk.Tk()
ROOT.withdraw()
company_json = {}
other_details = {}
# the input dialog
input_string = simpledialog.askstring(title="Ticker",
                                  prompt="Please enter ur tickers seperated by comma")
ticker  = input_string.split(",")
for tic in ticker : 
    url = f'https://fr.finance.yahoo.com/quote/{tic}?p={tic}'
    req = Request(url)
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    tick = {}
    for span in soup.findAll('span',
                         attrs={'class': 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'
                         }):
        tick['PRESENT_VALUE'] = span.text.strip()
    for div in soup.findAll('div', attrs={'class': 'D(ib) Va(t)'}):
        for span in div.findAll('span', recursive=False):
            tick['PRESENT_GROWTH'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
        for span in td.findAll('span', recursive=False):
            tick['PREV_CLOSE'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
        for span in td.findAll('span', recursive=False):
            tick['OPEN'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
        for span in td.findAll('span', recursive=False):
            tick['BID'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
        for span in td.findAll('span', recursive=False):
            tick['ASK'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
        for span in td.findAll('span', recursive=False):
            tick['DAYS_RANGE'] = span.text.strip()
    for td in soup.findAll('td',
                       attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
        for span in td.findAll('span', recursive=False):
            tick['FIFTY_TWO_WK_RANGE'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
        for span in td.findAll('span', recursive=False):
            tick['TD_VOLUME'] = span.text.strip()
    for td in soup.findAll('td',
                       attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'
                       }):
        for span in td.findAll('span', recursive=False):
            tick['AVERAGE_VOLUME_3MONTH'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
        for span in td.findAll('span', recursive=False):
            tick['MARKET_CAP'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
        for span in td.findAll('span', recursive=False):
            tick['BETA_3Y'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
        for span in td.findAll('span', recursive=False):
            tick['PE_RATIO'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
        for span in td.findAll('span', recursive=False):
            tick['EPS_RATIO'] = span.text.strip()
    for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'
                       }):
        tick['EARNINGS_DATE'] = []
        for span in td.findAll('span', recursive=False):
            tick['EARNINGS_DATE'].append(span.text.strip())
    for td in soup.findAll('td',
                       attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
        tick['DIVIDEND_AND_YIELD'] = td.text.strip()
    for td in soup.findAll('td',
                       attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
        for span in td.findAll('span', recursive=False):
            tick['EX_DIVIDEND_DATE'] = span.text.strip()
    for td in soup.findAll('td',
                       attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'
                       }):
        for span in td.findAll('span', recursive=False):
            tick['ONE_YEAR_TARGET_PRICE'] = span.text.strip()
    company_json[tic] = tick
    
    with open('data.json', 'w') as outfile:
         json.dump(company_json, outfile, indent=4)
    print(company_json)
    with open('output_file.html', 'wb') as file:
        file.write(html)
    print ('----------Extraction of data is complete. Check json file.----------')

{'AMZN': {'PRESENT_VALUE': '2\xa0401,10', 'PREV_CLOSE': '2\xa0410,39', 'OPEN': '2\xa0384,33', 'BID': '2\xa0402,71 x 900', 'ASK': '2\xa0404,90 x 1200', 'TD_VOLUME': '3\xa0092\xa0643', 'AVERAGE_VOLUME_3MONTH': '6\xa0027\xa0524', 'MARKET_CAP': '1,198T', 'PE_RATIO': '114,69', 'EPS_RATIO': '20,94', 'EARNINGS_DATE': ['23 juil. 2020', '27 juil. 2020'], 'DIVIDEND_AND_YIELD': 'S.O. (S.O.)', 'ONE_YEAR_TARGET_PRICE': '2\xa0645,67'}}
----------Extraction of data is complete. Check json file.----------
{'AMZN': {'PRESENT_VALUE': '2\xa0401,10', 'PREV_CLOSE': '2\xa0410,39', 'OPEN': '2\xa0384,33', 'BID': '2\xa0402,71 x 900', 'ASK': '2\xa0404,90 x 1200', 'TD_VOLUME': '3\xa0092\xa0643', 'AVERAGE_VOLUME_3MONTH': '6\xa0027\xa0524', 'MARKET_CAP': '1,198T', 'PE_RATIO': '114,69', 'EPS_RATIO': '20,94', 'EARNINGS_DATE': ['23 juil. 2020', '27 juil. 2020'], 'DIVIDEND_AND_YIELD': 'S.O. (S.O.)', 'ONE_YEAR_TARGET_PRICE': '2\xa0645,67'}, 'FB': {'PRESENT_VALUE': '225,46', 'PREV_CLOSE': '229,14', 'OPEN': '224,30', 'BI

In [186]:
import pandas as pd  
multi_data = pd.read_json("data.json") 
multi_data

,AMZN,FB,GOOG
PRESENT_VALUE,"2 401,10","225,46","1 416,73"
PREV_CLOSE,"2 410,39","229,14","1 417,84"
OPEN,"2 384,33","224,30","1 396,86"
BID,"2 402,71 x 900","226,36 x 1000","1 420,98 x 1200"
ASK,"2 404,90 x 1200","226,50 x 800","1 426,39 x 800"
TD_VOLUME,3 092 643,28 900 544,1 627 556
AVERAGE_VOLUME_3MONTH,6 027 524,27 101 270,2 417 262
MARKET_CAP,"1,198T","642,358B","967,933B"
PE_RATIO,"114,69","30,94","28,58"
EPS_RATIO,"20,94","7,29","49,57"


<h2>Web scrapping : Potential Improvements</h2>

Many improvement may be brought to this code.The data has to be cleaned to be exploited <p>
On of the improvements is to Retrieve historical data giving the user the freedom to fetch on a range of date.
Also build a function which take tickers , historical depth and the attributes needed or even Scheduling the Scrape Job